In [1]:
from datetime import timedelta
from rich import print
from dynaconf import Dynaconf
from arango import ArangoClient
from utinni import Context
from pathlib import Path

from tiro.plugins.utinni import TiroTSPump
from tiro.plugins.arango import ArangoAgent
from tiro.core import Scenario

conf = Dynaconf(settings_files=["./config/utinni_config.toml"])
scenario = Scenario.from_yaml(Path("./config/scenario.yaml"), Path("./config/use1.yaml"))

In [2]:
context = Context(**conf.on_request)
context.add_pump("tiro", TiroTSPump(scenario=scenario, **conf.influxdb))

In [3]:
table = context.tiro_table(".*Server%(CPU|Memory)Temperature", column="Server")

In [4]:
context.bind(start=-timedelta(hours=1),
             step=timedelta(minutes=10))

table["MemoryTemperature"].value

,b9c89dd0-c984-11ec-883c-0242c0a8c004,b9c89e70-c984-11ec-883c-0242c0a8c004,b9c89ed4-c984-11ec-883c-0242c0a8c004,b9c89fce-c984-11ec-883c-0242c0a8c004,b9c8a032-c984-11ec-883c-0242c0a8c004,b9c8a0f0-c984-11ec-883c-0242c0a8c004,b9c8a154-c984-11ec-883c-0242c0a8c004,b9c8a1ae-c984-11ec-883c-0242c0a8c004,b9c8a208-c984-11ec-883c-0242c0a8c004,b9c8a258-c984-11ec-883c-0242c0a8c004,...,b9c8dad4-c984-11ec-883c-0242c0a8c004,b9c8dba6-c984-11ec-883c-0242c0a8c004,b9c8dc0a-c984-11ec-883c-0242c0a8c004,b9c8dc6e-c984-11ec-883c-0242c0a8c004,b9c8dce6-c984-11ec-883c-0242c0a8c004,b9c8dd54-c984-11ec-883c-0242c0a8c004,b9c8ddcc-c984-11ec-883c-0242c0a8c004,b9c8de30-c984-11ec-883c-0242c0a8c004,b9c8de9e-c984-11ec-883c-0242c0a8c004,b9c8def8-c984-11ec-883c-0242c0a8c004
2022-05-01 18:30:00+00:00,23.843333,22.441667,21.258333,23.338333,21.393333,18.405,22.48,18.433333,18.495,20.853333,...,20.403333,15.524,23.676,17.594,21.314,24.386,20.410,20.348,27.776,17.248
2022-05-01 18:40:00+00:00,23.843333,22.441667,21.258333,23.338333,21.393333,18.405,22.48,18.433333,18.495,20.853333,...,20.403333,15.524,23.676,17.594,21.314,24.386,20.410,20.348,27.776,17.248
2022-05-01 18:50:00+00:00,23.843333,22.441667,21.258333,23.338333,21.393333,18.405,22.48,18.433333,18.495,20.853333,...,20.403333,15.524,23.676,17.594,21.314,24.386,20.410,20.348,27.776,17.248
2022-05-01 19:00:00+00:00,23.843333,22.441667,21.258333,23.338333,21.393333,18.405,22.48,18.433333,18.495,20.853333,...,20.403333,15.524,23.676,17.594,21.314,24.386,20.410,20.348,27.776,17.248
2022-05-01 19:10:00+00:00,23.843333,22.441667,21.258333,23.338333,21.393333,18.405,22.48,18.433333,18.495,20.853333,...,20.403333,15.524,23.676,17.594,21.314,24.386,20.410,20.348,27.776,17.248
2022-05-01 19:20:00+00:00,23.843333,22.441667,21.258333,23.338333,21.393333,18.405,22.48,18.433333,18.495,20.853333,...,20.403333,15.524,23.676,17.594,21.314,24.386,20.410,20.348,27.776,17.248
2022-05-01 19:30:00+00:00,19.910000,22.790000,25.700000,18.420000,19.980000,22.980,23.53,27.870000,26.540,13.540000,...,15.360000,27.195,16.765,19.650,27.315,18.140,17.765,22.165,25.015,19.970


In [5]:
gdb_client = ArangoAgent(scenario, "tiro_test", "scenario",
                         ArangoClient(hosts="http://localhost:8529"),
                         auth_info=dict(password="arangodb_password"))

In [6]:
data = gdb_client.capture_status(pattern=".*Server%(CPU|Memory)Temperature")
print(data)

{
    'Room': {
        'b9c89ae2-c984-11ec-883c-0242c0a8c004': {
            'Rack': {
                'b9c89c2c-c984-11ec-883c-0242c0a8c004': {
                    'Server': {
                        'b9c89dd0-c984-11ec-883c-0242c0a8c004': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 19.91,
                                    'unit': '°C',
                                    'timestamp': '2022-05-01T19:30:23.829183'
                                }
                            }
                        },
                        'b9c89e70-c984-11ec-883c-0242c0a8c004': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 22.79,
                                    'unit': '°C',
                                    'timestamp': '2022-05-01T19:30:23.993404'
                                }
                            }
                        },
                        'b9c89ed4-c984-11ec-883c-0242c0a8c004': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 25.7,
                                    'unit': '°C',
                                    'timestamp': '2022-05-01T19:30:23.654708'
                                }
                            }
                        }
                    }
                },
                'b9c89f42-c984-11ec-883c-0242c0a8c004': {
                    'Server': {
                        'b9c89fce-c984-11ec-883c-0242c0a8c004': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 18.42,
                                    'unit': '°C',
                                    'timestamp': '2022-05-01T19:30:23.096052'
                                }
                            }
                        },
                        'b9c8a032-c984-11ec-883c-0242c0a8c004': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 19.98,
                                    'unit': '°C',
                                    'timestamp': '2022-05-01T19:30:23.735935'
                                }
                            }
                        }
                    }
                },
                'b9c8a08c-c984-11ec-883c-0242c0a8c004': {
                    'Server': {
                        'b9c8a0f0-c984-11ec-883c-0242c0a8c004': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 22.98,
                                    'unit': '°C',
                                    'timestamp': '2022-05-01T19:30:24.708767'
                                }
                            }
                        },
                        'b9c8a154-c984-11ec-883c-0242c0a8c004': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 23.53,
                                    'unit': '°C',
                                    'timestamp': '2022-05-01T19:30:24.491690'
                                }
                            }
                        },
                        'b9c8a1ae-c984-11ec-883c-0242c0a8c004': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 27.87,
                                    'unit': '°C',
                                    'timestamp': '2022-05-01T19:30:24.148626'
                                }
                            }
                        },
                        'b9c8a208-c984-11ec-883c-0242c0a8c004': {
                 